<a href="https://colab.research.google.com/github/leonamcassemir0/Machine-Learning/blob/main/Classifica%C3%A7%C3%A3o_de_texto_com_avalia%C3%A7%C3%B5es_de_filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando bibliotecas

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.17.0


# Baixando o IMDB

In [2]:
imdb = keras.datasets.imdb

(train_data, train_label), (test_data, test_label) = imdb.load_data(num_words=1000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


O argumento ***num_words=10000*** mantém as 10000 palavras mais frequentes no conjunto de treinamento. As palavras mais raras são descartadas para preservar o tamanho dos dados de forma maleável.

# Explore o dados

In [3]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_label)))
print(train_data[0])
len(train_data[0]), len(train_data[1])

Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]


(218, 189)

# Convertendo os índices para palavras

In [4]:
# Um dicionário mapeando palavras em índices inteiros
word_index = imdb.get_word_index()

# Os primeiros índices são reservados
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


***.get_word_index()*** é um método para obter um dicionário que mapeia palavras para índices inteiros com base em sua frequência no conjunto de dados do IMDb.

***.items()*** retorna uma visão de objetos do dicionário, que contém pares de chave-valor como tuplas.  
*   Isso é útil quando você precisa iterar sobre um dicionário acessando tanto as chaves quanto os valores ao mesmo tempo.

***.get(key, default_value)*** permite especificar um valor padrão que será retornado caso a chave não exista no dicionário.

***.join(iterable)*** é usado para concatenar uma lista de strings em uma única string, usando um separador especificado.

### Podemos verificar a análise do primeiro filme

In [5]:
decode_review(train_data[0])

"<START> this film was just brilliant casting <UNK> <UNK> story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> <UNK> as myself so i loved the fact there was a real <UNK> with this film the <UNK> <UNK> throughout the film were great it was just brilliant so much that i <UNK> the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you <UNK> at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they have done don't y

# Prepare os dados

In [6]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='pre',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='pre',
                                                       maxlen=256)

print(train_data[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   1  14  22  16  43 530 973   2   2  65 458   2  66   2   4 173
  36 256   5  25 100  43 838 112  50 670   2   9  35 480 284   5 150   4
 172 112 167   2 336 385  39   4 172   2   2  17 546  38  13 447   4 192
  50  16   6 147   2  19  14  22   4   2   2 469   4  22  71  87  12  16
  43 530  38  76  15  13   2   4  22  17 515  17  12  16 626  18   2   5
  62 386  12   8 316   8 106   5   4   2   2  16 480  66   2  33   4 130
  12  16  38 619   5  25 124  51  36 135  48  25   2  33   6  22  12 215
  28  77  52   5  14 407  16  82   2   8   4 107 117   2  15 256   4   2
   7   2   5 723  36  71  43 530 476  26 400 317  46   7   4   2   2  13
 104  88   4 381  15 297  98  32   2  56  26 141   6 194   2  18   4 226
  22  21 134 476  26 480   5 144  30   2  18  51  36  28 224  92  25 104
   4 226  65  16  38   2  88  12  16 283   5  16   



***pad_sequences( )*** esta função ajusta (preenche ou corta) as sequências de inteiros para que todas tenham o mesmo comprimento.

* ***value=word_index[ ]***: é um token especial usado para preenchimento, e o valor correspondente a este token no dicionário word_index é utilizado para preencher as sequências.
* **padding='post'**:Indica que o preenchimento deve ser adicionado ao final das sequências.
* **maxlen=256**: Define o comprimento máximo das sequências após o preenchimento ou corte.

### Resultado:
Após a execução deste código:
- Todas as sequências em `train_data` e `test_data` terão um comprimento de 256.
- As sequências que originalmente tinham menos de 256 elementos serão preenchidas com o valor correspondente a `"<PAD>"` no final.
- As sequências que eram mais longas serão cortadas, mantendo apenas os primeiros 256 elementos.


# Construindo o modelo

In [7]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

***.GlobalAveragePooling1D():*** Calcula a média dos vetores de embedding ao longo do tempo, transformando os vetores de palavra em um único vetor de comprimento fixo (16 no caso).



## Embedding

Essa camada mapeia cada palavra do vocabulário para um vetor de dimensão 16.
* Essencialmente, a camada converte as palavras em representações vetoriais de menor dimensão, o que facilita o processamento pelo modelo.
* Each word in the vocabulary is mapped to a unique vector of floating-point numbers.
* The dimensions of these vectors are much smaller than the size of the vocabulary.

### Benefícios
* ***Dimensionality Reduction:*** Instead of using one-hot encoding, we use a much smaller 16-dimensional vector.
* ***Semantic Meaning:*** Words with similar meanings tend to have similar embeddings, capturing semantic relationships.
* ***Efficiency:*** Embeddings are more computationally efficient than one-hot encoding for large vocabularies.


# Função loss e optmizer

In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Conjunto de validação

In [14]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_label[:10000]
partial_y_train = train_label[10000:]

# Treine o modelo

In [17]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/embedding_1/GatherV2 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local